#### Stage 1
# Flow Data - Spatial Overview
with folium maps

##### Generating Flow Maps
This notebook includes the code to generate **visualization number 1** from **flowdf.csv** with help of the **celldf.csv** containing the cell geometries.
The mapping function first filters and aggregates the flow dataset according to the chosen temporal filters, averaging quantities over different days and summing over different hours.
Flows with at least five movements are then plotted on a *folium* map on top of polygons representing the mobile network cells.
The lines are coloured according to the chosen mode and their widths are calculated using the number of movements within the chosen mode and normalized to either the mean of all plotted lines' widths or that of each segment throughout the whole dataset in case "normalize" is checked in the filter options.
Tooltips are added to the cell-polygons stating the cell-ID and to the flow lines containing the number of movements the line represents as well as the chosen mode and the flow-ID giving the two cell-IDs that are connected by the flow line.

##### Versions of the used packages:
- pandas: 0.24.2
- numpy: 1.16.4
- folium: 0.11.0
- ipywidgets: 7.5.1

In [1]:
import pandas as pd
from ast import literal_eval
import folium
import numpy as np
import ipywidgets as pyw

In [2]:
centroidBounds = [[[48.04821822889212, 11.67438639612782]],[[48.39922017377597, 11.75558830832081]]]
mapbox_access_token = r'MAPBOX_ACCESS_TOKEN'
tileset_ID_str = 'streets-v11'
# tileset_ID_str = 'outdoors-v11'
# tileset_ID_str = 'light-v10'
# tileset_ID_str = 'dark-v10'
# tileset_ID_str = 'satellite-v9'
# tileset_ID_str = 'satellite-streets-v11'
tilesize_pixels = '256'
#Tiles = 'OpenStreetMap'
#Tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
Tiles = f"https://api.mapbox.com/styles/v1/klavere/ckd8wc3hf06we1imiypydrqcf/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
MapboxAttribution = '<a href="https://www.mapbox.com/about/maps/">© Mapbox</a> | <a href="http://www.openstreetmap.org/about/">© OpenStreetMap</a> | <a href="https://www.mapbox.com/map-feedback/#/-74.5/40/10">Improve this map</a>'

In [3]:
def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def getCoordsBack(coords):
    return literal_eval(coords);

In [4]:
celldf = csvtodf_SC('celldf')
celldf['polyCoords'] = celldf.apply(lambda row: getCoordsBack(row.polyCoords), axis = 1)
celldf['centroidCoords'] = celldf.apply(lambda row: getCoordsBack(row.centroidCoords), axis = 1)
celldf = celldf.reindex(columns = ['cellID', 'polyCoords', 'centroidCoords'])
celldf.head()

,cellID,polyCoords,centroidCoords
0,1,"[[48.14605999999999, 11.510794], [48.146254999...","[48.15396752884293, 11.51032914199828]"
1,2,"[[48.23591199999999, 11.635515], [48.237070000...","[48.22263647015427, 11.62774480807693]"
2,3,"[[48.14900599999999, 11.696217], [48.153606000...","[48.13564057367491, 11.70093894926043]"
3,4,"[[48.12879600000001, 11.541556], [48.125282, 1...","[48.13194111905815, 11.54797042679206]"
4,5,"[[48.18833000000001, 11.615821], [48.193665, 1...","[48.19669108307331, 11.6113147037113]"


In [5]:
flowdf = csvtodf_SC('flowdf')
flowdf['flowCoords'] = flowdf.apply(lambda row: getCoordsBack(row.flowCoords), axis = 1)
flowdf.head()

,flowID,dayInt,weekday,dayType,hour,moves,privat,public,Rail,UBahn,Tram,Bus,flowCoords,meanmoves,meanprivat,meanpublic,meanRail,meanUBahn,meanTram,meanBus
0,100_101,0,Monday,MonThu,4,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.39922017377597, 11.75558830832081], [48.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100_101,0,Monday,MonThu,5,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.22013108151531, 11.52767333862167], [48.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100_101,0,Monday,MonThu,15,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.39922017377597, 11.75558830832081], [48.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100_101,1,Tuesday,MonThu,4,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.39922017377597, 11.75558830832081], [48.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100_101,1,Tuesday,MonThu,5,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.39922017377597, 11.75558830832081], [48.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## map flows

maps flows for each mode, absolute or wrt mean of all data for the respective segment

In [6]:
days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dayTypes = ['', 'MonThu', 'Fri', 'Weekend']

modes = ['moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus']
meanModes = ['']*7
for i in range(0,7):
    meanModes[i] = 'mean'+modes[i]

modesColor = {'privat':'#999999',#'#BFBFBF',
              'Rail':'#4daf4a',
              'UBahn':'#377eb8',
              'Tram':'#e41a1c',
              'Bus':'#984ea3',
              'public':'#ff7f00', # '#ff7f00',
              'moves':'#a65628'} # '#FFFFFF'}
# including ColorBrewer Colors from: https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=9

d = {'flowCoords':['first']}
modesAggMean = dict((key, ['mean']) for key in modes+meanModes)
modesAggSum = dict((key, ['sum']) for key in modes+meanModes)

In [7]:
flowdf.head(2)

,flowID,dayInt,weekday,dayType,hour,moves,privat,public,Rail,UBahn,Tram,Bus,flowCoords,meanmoves,meanprivat,meanpublic,meanRail,meanUBahn,meanTram,meanBus
0,100_101,0,Monday,MonThu,4,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.39922017377597, 11.75558830832081], [48.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100_101,0,Monday,MonThu,5,1,1.0,0.0,0.0,0.0,0.0,0.0,"[[48.22013108151531, 11.52767333862167], [48.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def mapFlowsMode(mode='moves', day='', dayType='', hours=(0,24), normalize=False):
    
    # aggregate flowdf according to filter choices for day, dayType and hour. mean between days and dayTypes, sum over hours.
    d.update(modesAggMean)
    if (day):
        mapdf = flowdf[(flowdf.weekday == day)].groupby(['flowID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    elif (dayType):
        mapdf = flowdf[(flowdf.dayType == dayType)].groupby(['flowID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    else:
        mapdf = flowdf.groupby(['flowID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    
    d.update(modesAggSum)
    mapdf = mapdf[(mapdf.hour.between(hours[0]-1,hours[1],inclusive=False))]
    mapdf = mapdf.groupby(['flowID']).agg(d).copy().reset_index()
    mapdf.columns = mapdf.columns.get_level_values(0)

    # initialize map with cell polygons
    m = folium.Map(tiles = Tiles,
                   attr = MapboxAttribution)
    m.fit_bounds(centroidBounds)
    for index, row in celldf.iterrows():
        line_i = folium.Polygon(locations=row.polyCoords,
                                color = '#ff7f00',#'#FBF9F7',
                                weight = 1,
                                fill = True,
                                opacity = 0.5,
                                fill_opacity = 0.02,
                                tooltip = 'Cell'+str(row.cellID)).add_to(m)
    
    # add for loop for multiple modes (maybe)
    
    # put flows on map
    color = modesColor[mode]
    mapdf = mapdf[(mapdf[mode] >= 5)] # exclude flows < 5
    M = 2*np.mean(mapdf[mode]) # mean for linewidths if normalized=False
    
    for index, row in mapdf.iterrows():
        if (normalize):
            M = row[str('mean'+mode)] # get flowID-specific mean if noramlized=True
            if (M<5): M = np.nan
            tooltip = str(int(row[str(mode)]))+' moves<br>'+'mode = '+str(mode)+'<br>'+str(np.round(row[mode]/M, decimals=1))+' times normal amount<br>'+'flow-ID '+str(row.flowID)
        else: tooltip = str(int(row[str(mode)]))+' moves<br>'+'mode = '+str(mode)+'<br>'+'flow-ID '+str(row.flowID)
        weight = 3*row[mode]/M
        if (weight):
            line_i = folium.PolyLine(locations = row.flowCoords,
                                     color = color,
                                     weight = weight,
                                     opacity = 0.6,
                                     tooltip = tooltip)
            m.add_child(line_i)
    return m;

In [9]:
range_slider_h = pyw.IntRangeSlider(
    value=[0,24],
    min=0, max=24, step=1,
    description='Time of day')

In [10]:
pyw.interact_manual(mapFlowsMode,
                    mode = modes,
                    day = days,
                    dayType = dayTypes,
                    hours=range_slider_h)

interactive(children=(Dropdown(description='mode', options=('moves', 'privat', 'public', 'Rail', 'UBahn', 'Tra…

<function __main__.mapFlowsMode(mode='moves', day='', dayType='', hours=(0, 24), normalize=False)>